<a href="https://colab.research.google.com/github/eruru0411/git-test/blob/master/210226_cat_dog_like_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
colab_path = '/content/drive/MyDrive/Colab Notebooks/data/cat_dog_2/datasets/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from keras.datasets import mnist
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten

import os 
from keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator

np.random.seed(121)
tf.random.set_seed(121)

import warnings
warnings.filterwarnings(action='ignore')

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [ ]:
def CHART_PLOT_HISTORY(history):
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch

    plt.figure(figsize=(8,12))

    plt.subplot(2,1,1)
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.plot(hist['epoch'], hist['accuracy'],    label='acc')
    plt.plot(hist['epoch'], hist['val_accuracy'],label = 'val_acc')
    plt.legend()

    plt.subplot(2,1,2)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.plot(hist['epoch'], hist['loss'],    label='loss')
    plt.plot(hist['epoch'], hist['val_loss'],label = 'val_loss')
    plt.legend()
    plt.show()

### 데이터 불러오기

In [ ]:
import zipfile

zip_file = zipfile.ZipFile(colab_path + 'train.zip')
zip_file.extractall(colab_path + 'train/')

In [ ]:
train_file_list = os.listdir(colab_path + "train")
print("train img:", len(train_file_list))

test_file_list = os.listdir(colab_path + "test")
print("test img:", len(test_file_list))

### sampling

In [ ]:
EPOCH = 35
BATCH = 250

# train_file_sampling = train_file_list[:1000] + train_file_list[-1000: ]
# del train_file_list
# train_file_list = train_file_sampling
#
# test_file_sampling = test_file_list[:200]
# del test_file_list
# test_file_list = test_file_sampling


### train 이미지 배열화

In [ ]:
img_arr_list = []
label_list = []
for fname in train_file_list:
    path = os.path.join('./dataset/train/'+fname)
    img_object = load_img(path, target_size=(128,128))
    img_arr = img_to_array(img_object) / 255.0
    img_arr_list.append(img_arr)
    if fname.split(".")[0] == 'cat':
        label_list.append(0)   #cat.1.jpg
    else :
        label_list.append(1)  # dog.1.jpg
# X_train : (60000, 28(W), 28(H), 1(Channel))
print(np.array(img_arr_list).shape)   #(2000, 128, 128, 3)
print(np.array(label_list).shape)     #(2000,)

X = np.array(img_arr_list)
y = np.array(label_list)

### train 8:2

In [ ]:
train_idx = np.round(len(train_file_list)*0.8,decimals=0)
test_idx  = np.round(len(test_file_list)*0.8,decimals=0)
X_train = X[:train_idx]
X_val   = X[train_idx:]
y_train = y[:test_idx]
y_val   = y[test_idx:]

### test 이미지 배열화

In [ ]:
img_arr_list = []
for fname in test_file_list:
    path = os.path.join('./dataset/test/'+fname)
    img_object = load_img(path, target_size=(128,128))
    img_arr = img_to_array(img_object) / 255.0
    img_arr_list.append(img_arr)

# X_train : (60000, 28(W), 28(H), 1(Channel))
print(np.array(img_arr_list).shape)   #(200, 128, 128, 3)
X_test = np.array(img_arr_list)

### 이하 MNIST 동일

In [ ]:
model = Sequential()
model.add(Conv2D(filters=20,kernel_size=(4,4), input_shape=(128, 128, 3), activation='relu')    )
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(filters=40, kernel_size=(3,3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))  #Active map : 6*6*40
model.add(Flatten())  #input_dim=
model.add(Dense(64, activation='relu'))
model.add(Dense(2, activation='softmax'))  #출력결과 0~9숫자
print(model.summary())   #total params : 72,294

In [ ]:
from keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10)
model.compile(optimizer="adam",   #GD(경사하강) SGD momentum ada.. adam
              loss='sparse_categorical_crossentropy', #손실함수 cost loss
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train,y_train,
          epochs = EPOCH,
          batch_size=BATCH,
          validation_data=(X_val, y_val),
          callbacks=[early_stop])

In [ ]:
print('acc: {:.4f}, loss: {:.4f}'.format(model.evaluate(X_val, y_val)[1], model.evaluate(X_val, y_val)[0]))

# pred = model.predict(X_test) #[[0.35  0.65] .... 200n]
# print(np.argmax(pred))       #[[1] ... 200n]

CHART_PLOT_HISTORY(history)